In [1]:
import pandas as pd
import numpy as np

In [2]:
import sys
from os import path        
sys.path.append('../scripts')
from file_management import tif_data
%load_ext autoreload
%autoreload 2

In [12]:
raw = tif_data('../build/population-within-1km.tif')
raw

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
raw = tif_data('../build/data/population/JRC_1K_POP_2018.tif', replace_data=)
raw.data * (1-raw.mask)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [38]:
len(raw)

4472

In [36]:
sum(sum(raw))

4730542699999.17

In [7]:
def disamenity_costs(radius_from, radius_to, scenario = 'low') -> float:
    # Calculates the disamenity costs in € per person, per annum, per MW installed wind capacity
    # radius from is the inner radius in km and radius_to the outer radius of the assessed area.

    if scenario == 'low':
        cost_slope = -31
        max_distance = 4
    elif scenario == 'high':
        cost_slope = -64
        max_distance = 8
    else:
        print('Not a valid scenario name')

    assert (radius_to > 0) & (radius_from >= 0), 'radius must be positive'
    assert radius_to > radius_from, 'radius_to must be larger than radius_from'
    assert radius_to <= max_distance, f'radius_to exceeds the maximal distance'


    # costs per household, per year, and per windpark
    area_weighted_costs = \
        cost_slope/2 * (
            radius_to**2 * (2*np.log(radius_to) -1)
            -(radius_from**2 * (2*np.log(radius_from) -1)) # TODO: this log(radus_from) does not allow to calculate the the disamenity for an interval starting at zero
        ) / (
            radius_to**2
            -radius_from**2
        ) - cost_slope * np.log(max_distance)

    # Further assumptions
    GBP_EUR = 0.86
    people_household = 2
    turbines_park = 12.5
    MW_turbine = 2

    return(area_weighted_costs / GBP_EUR / people_household / turbines_park / MW_turbine)


In [17]:
disamenity_costs(0.2,1)

1.3115411137478117

In [18]:
disamenity_costs(2,4)

0.1938948635863698